In [1]:
%pip install datasets
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('yelp_review_full')

Note: you may need to restart the kernel to use updated packages.


/Users/caoyun/miniconda3/envs/ml-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(dataset)
print(dataset['train'][:5])
df = dataset['train'].to_pandas()
print(df.head())

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})
{'label': [4, 1, 3, 3, 0], 'text': ["dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.", "Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It u

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("LiYuan/amazon-review-sentiment-analysis")

model = AutoModelForSequenceClassification.from_pretrained("LiYuan/amazon-review-sentiment-analysis")


tokenizer_config.json: 100%|██████████| 556/556 [00:00<00:00, 32.5kB/s]
vocab.txt: 100%|██████████| 872k/872k [00:00<00:00, 4.33MB/s]
tokenizer.json: 100%|██████████| 2.56M/2.56M [00:00<00:00, 28.8MB/s]
special_tokens_map.json: 100%|██████████| 112/112 [00:00<00:00, 53.4kB/s]
config.json: 100%|██████████| 1.23k/1.23k [00:00<00:00, 655kB/s]
/Users/caoyun/miniconda3/envs/ml-env/lib/python3.8/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
pytorch_model.bin: 100%|██████████| 670M/670M [00:29<00:00, 22.7MB/s] 


In [29]:
from transformers import pipeline

# Create a sentiment analysis pipeline using the specified model
sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, truncation=True)

sample = df.head(1000)
results = sentiment_pipeline(sample['text'].tolist())
# max_length = 512  # This is the typical max length for transformer models

# Apply the sentiment analysis pipeline to truncated text
# results = [sentiment_pipeline(text[:max_length]) for text in sample['text']]

# Display the results
# for text, prediction in zip(sample['text'], results):
#     print(f"Text: {text}\nScore: {prediction[0]['label']}, Sentiment analysis: {prediction[0]['score']}\n")

In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("LiYuan/amazon-review-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("LiYuan/amazon-review-sentiment-analysis")

LiYuan_pipeline = pipeline(model=model, tokenizer=tokenizer, task="sentiment-analysis")

# Initialize an empty list to store the outputs
LiYuan_output = []

# Iterate over the Yelp reviews
for review in dataset['train']['text']:
    # Encode the review
    encoded_review = tokenizer.encode(review)

    # Check if the encoded review needs to be truncated
    if len(encoded_review) > 500:
        # Truncate the encoded review to the first 512 tokens
        encoded_review = encoded_review[:500]

    # Perform sentiment analysis and store the output
    LiYuan_output.append(LiYuan_pipeline(tokenizer.decode(encoded_review)))

import pandas as pd
# convert the output to a dataframe including the text
df = pd.DataFrame(LiYuan_output)
df["text"] = dataset['train']['text']
print(df)


In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.utils.data import DataLoader

# If a GPU is available, move the model to the GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained("LiYuan/amazon-review-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("LiYuan/amazon-review-sentiment-analysis").to(device)

LiYuan_pipeline = pipeline(model=model, tokenizer=tokenizer, task="sentiment-analysis", device=0 if torch.cuda.is_available() else -1)

# Initialize an empty list to store the outputs
LiYuan_output = []

# Define the batch size
batch_size = 16

# Create a DataLoader for parallelizing the data loading
data_loader = DataLoader(dataset['train']['text'], batch_size=batch_size)



# Iterate over the Yelp reviews in batches
for batch_reviews in data_loader:
    # Iterate over the Yelp reviews
    for review in batch_reviews:
        # Encode the review
        encoded_review = tokenizer.encode(review)

        # Check if the encoded review needs to be truncated
        if len(encoded_review) > 500:
            # Truncate the encoded review to the first 512 tokens
            encoded_review = encoded_review[:500]

        # Perform sentiment analysis and store the output
        LiYuan_output.append(LiYuan_pipeline(tokenizer.decode(encoded_review)))
    # # Truncate the reviews to the first 500 tokens
    # truncated_reviews = [review[:500] for review in batch_reviews]

    # # Perform sentiment analysis and store the output
    # LiYuan_output.extend(LiYuan_pipeline(truncated_reviews))

import pandas as pd
# convert the output to a dataframe including the text
df = pd.DataFrame(LiYuan_output)
df["text"] = dataset['train']['text']
print(df)


In [ ]:
from transformers import pipeline
import pandas as pd

# Create the model pipeline
model_pipeline = pipeline(model="distilbert-base-uncased-finetuned-sst-2-english")

# Yelp reviews
yelp_reviews = dataset['train']['text']

# Initialize an empty list to store the outputs
model_output = []

# Iterate over the Yelp reviews
for review in yelp_reviews:
    # Check if the review needs to be truncated
    if len(tokenizer.encode(review)) > 512:
        # Truncate the review to the first 512 tokens
        review = tokenizer.decode(tokenizer.encode(review)[:512])

    # Perform sentiment analysis and store the output
    model_output.append(model_pipeline(review))

# Convert the output to a dataframe including the text
df = pd.DataFrame(model_output)
df["text"] = yelp_reviews
print(df)


In [ ]:
# --------------------
# Import pipeline
# --------------------
from transformers import pipeline

# --------------------
# Create the task pipeline
# --------------------
task_pipeline = pipeline(task="sentiment-analysis")

# --------------------
# Create the model pipeline
# --------------------
model_pipeline = pipeline(model="siebert/sentiment-roberta-large-english")
# Yelp reviews
yelp_reviews = dataset['train']['text']

# Predict the sentiment of multiple sentences
task_output = task_pipeline(yelp_reviews)
model_output = model_pipeline(yelp_reviews)

import pandas as pd
# convert the output to a dataframe including the text
df = pd.DataFrame(task_output)
df["text"] = yelp_reviews
print(df)